In [1]:
import pandas as pd
import pickle
from pyspark import ml
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SparkSession

# pd.set_option('display.max_colwidth', None)
spark = SparkSession.builder.appName('ml-anom').getOrCreate()

23/05/01 00:57:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [ ]:
processed_input = "/data/log_data.csv"

input_df = spark.read.csv(processed_input,header='true')
input_df.printSchema()

In [ ]:
# input_df.show()

In [ ]:
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import col

df2 = input_df.withColumn("is_private",col("is_private").cast(DoubleType())) \
.withColumn("is_root",col("is_root").cast(DoubleType())) \
.withColumn("is_failure",col("is_failure").cast(DoubleType())) \
.withColumn("time_since_last_failure_of_same_type",col("time_since_last_failure_of_same_type").cast(DoubleType())) \
.withColumn("failure_count_in_last_15_mins",col("failure_count_in_last_15_mins").cast(DoubleType())) \
.withColumn("failure_count_in_last_30_mins",col("failure_count_in_last_30_mins").cast(DoubleType())) \
.withColumn("failure_count_in_last_60_mins",col("failure_count_in_last_60_mins").cast(DoubleType())) \
.withColumn("label_auth_failure",col("label_auth_failure").cast(DoubleType())) \
.withColumn("label_break_in_attempt",col("label_break_in_attempt").cast(DoubleType())) \
.withColumn("label_connection_closed",col("label_connection_closed").cast(DoubleType())) \
.withColumn("label_disconnect",col("label_disconnect").cast(DoubleType())) \
.withColumn("label_failed_password",col("label_failed_password").cast(DoubleType())) \
.withColumn("label_invalid_user",col("label_invalid_user").cast(DoubleType())) \
.withColumn("label_no_label",col("label_no_label").cast(DoubleType())) \
.withColumn("label_no_identification",col("label_no_identification").cast(DoubleType())) \
.withColumn("class",col("class").cast(DoubleType()))
# df2.show()


In [ ]:
df2 = df2.drop("timestamp", "process_id", "username", "ip", "time_since_last_failure")
# df2.show()

In [ ]:
df2

In [ ]:
from pyspark.ml.feature import VectorAssembler

vecCols = ['is_private', 'is_root', 'is_failure', 'time_since_last_failure_of_same_type', 'failure_count_in_last_15_mins',
       'failure_count_in_last_30_mins', 'failure_count_in_last_60_mins','label_auth_failure', 'label_break_in_attempt',
       'label_connection_closed', 'label_disconnect', 'label_failed_password',
       'label_invalid_user', 'label_no_label', 'label_no_identification']
assembler = VectorAssembler(inputCols=vecCols, outputCol="vectors")
df2 = assembler.transform(df2)

In [ ]:
train, test = df2.randomSplit([0.7, 0.3], seed = 2018)

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

model = RandomForestClassifier(featuresCol = 'vectors', labelCol = 'class')
ranF = model.fit(train)

In [ ]:
# pickle.dump(model, open('ml_py_model.pkl', 'wb'))
ranF.write().overwrite().save('ml_py_model.pkl')

In [ ]:
from pyspark.ml.classification import RandomForestClassificationModel

model = RandomForestClassificationModel.load('ml_py_model.pkl')
pred = model.transform(test)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

eval = MulticlassClassificationEvaluator(labelCol="class", predictionCol="prediction")
accuracy = eval.evaluate(pred)
print("Accuracy = %s" % (accuracy))

In [ ]:
test.head()

In [2]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassificationModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# All columns are present. This is the data that comes from the "data_parser"
cust_col = ['timestamp','process_id','username','ip','is_private','is_root','is_failure','time_since_last_failure','time_since_last_failure_of_same_type','failure_count_in_last_15_mins','failure_count_in_last_30_mins',
            'failure_count_in_last_60_mins','label_auth_failure','label_break_in_attempt','label_connection_closed','label_disconnect','label_failed_password','label_invalid_user','label_no_label','label_no_identification','class']

# First 2 lines of from the log_data.csv file, including class
lst = [[1670673346.0,24200,'root','173.234.31.186',0,0,1,0.0,0.0,1,1,1,0,1,0,0,0,0,0,0,1], [1670673346.0,24200,'webmaster','173.234.31.186',0,0,1,0.0,0.0,2,2,2,0,0,0,0,0,1,0,0,0]]


# Create a spark dataframe and drop unwanted values
spark_df = spark.createDataFrame(lst, cust_col)
spark_df = spark_df.drop("timestamp", "process_id", "username", "ip", "time_since_last_failure")


# Vectorise dataframe
vecCols_cust = ['is_private', 'is_root', 'is_failure', 'time_since_last_failure_of_same_type', 'failure_count_in_last_15_mins', 'failure_count_in_last_30_mins', 'failure_count_in_last_60_mins','label_auth_failure', 
                'label_break_in_attempt', 'label_connection_closed', 'label_disconnect', 'label_failed_password', 'label_invalid_user', 'label_no_label', 'label_no_identification']
assembler_cust = VectorAssembler(inputCols=vecCols_cust, outputCol="vectors")
spark_df = assembler_cust.transform(spark_df)

# Import model from file and run the transform(i.e., predict) method
model_cust = RandomForestClassificationModel.load('ml_py_model.pkl')
pred_cust = model_cust.transform(spark_df)


# Evaluate accuracy of custom_prediction
eval_cust = MulticlassClassificationEvaluator(labelCol="class", predictionCol="prediction")
accuracy_cust = eval_cust.evaluate(pred_cust)
print("Accuracy = %s" % (accuracy_cust))

Accuracy = 1.0
